## Kansas City Crime and Property Data (2015-2020)
The following analysis uses Kansas City, MO Data for Crime and Property Violations in 2015 and 2020.

All crime and property violation data was gathered from [OpenData KC](https://data.kcmo.org/) on December 5, 2020.

All tax and income data was gathered from [IRS Statistics](https://www.irs.gov/statistics/) on December 6, 2020.

## Questions
- What are the comparisons between income, crime, and property violation across zip code?
- What are the recent changes in rate of crime and property violations?
- What contributes to crime rate? property violation rate?

In [20]:
# import necessary libraries and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from IPython.display import display
pd.options.display.max_columns = None
%matplotlib inline

## Gather Data

### Initial Data Import
- Rename headers that do not align between years due to misnaming (Reported_Time and Reported Time, From Time and From_Time, Firearm Used Flag, Location 1 and Location, etc) to provide a single variable across the entire dataframe

In [2]:
crime_2010 = pd.read_csv('kc_crime_2010.csv')
crime_2010.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2011 = pd.read_csv('kc_crime_2011.csv')
crime_2011.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2012 = pd.read_csv('kc_crime_2012.csv')
crime_2012.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2013 = pd.read_csv('kc_crime_2013.csv')
crime_2013.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2014 = pd.read_csv('kc_crime_2014.csv')
crime_2014.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2015 = pd.read_csv('kc_crime_2015.csv')
crime_2015.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2016 = pd.read_csv('kc_crime_2016.csv')
crime_2016.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2017 = pd.read_csv('kc_crime_2017.csv')
crime_2017.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2018 = pd.read_csv('kc_crime_2018.csv')
crime_2018.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 
crime_2019 = pd.read_csv('kc_crime_2019.csv')
crime_2019.rename(columns = {"From Time": "From_Time","To Time": "To_Time", "Reported Time": "Reported_Time", "Location 1": "Location", "Firearm Used Flag  ": "Firearm Used Flag"},  
          inplace = True) 

### Combine all crime dataframes together

In [3]:
crime_frames = [crime_2010, crime_2011, crime_2012, crime_2013, crime_2014, crime_2015, crime_2016, crime_2017, crime_2018, crime_2019]
crimes = pd.concat(crime_frames, ignore_index=True)

### Test Import Data for Consistency
Ensure that the columns translated over correctly

In [4]:
print(crimes.shape)
print(crimes.columns)
crimes.head()

(1234605, 26)
Index(['Report_No', 'Reported_Date', 'Reported_Time', 'From_Date', 'From_Time',
       'To_Date', 'To_Time', 'Offense', 'IBRS', 'Description', 'Beat',
       'Address', 'Zip Code', 'City', 'Rep_Dist', 'Area', 'DVFlag', 'Invl_No',
       'Involvement', 'Race', 'Sex', 'Age', 'Location', 'Firearm Used Flag',
       'Latitude', 'Longitude'],
      dtype='object')


,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,...,DVFlag,Invl_No,Involvement,Race,Sex,Age,Location,Firearm Used Flag,Latitude,Longitude
0,100048265,06/27/2010,13:46,06/27/2010,13:40,NaN,NaN,2655,90J,Trespassing,...,U,1.0,VIC,NaN,NaN,NaN,"100 13 ST\nKANSAS CITY, MO\n(39.19607633300046...",N,NaN,NaN
1,100043775,06/11/2010,21:00,06/10/2010,10:00,NaN,NaN,650,23G,Stealing Auto Parts/,...,U,1.0,VIC,B,M,40.0,NaN,N,NaN,NaN
2,100030602,04/27/2010,22:12,04/27/2010,19:00,NaN,NaN,1120,26A,Fraud,...,U,1.0,VIC,W,F,59.0,"600 NORTON AV\nKANSAS CITY, MO 64124\n(39.1056...",N,NaN,NaN
3,100000611,01/03/2010,19:41,01/03/2010,19:41,NaN,NaN,1120,26A,Fraud,...,U,2.0,VIC,NaN,NaN,NaN,"25 GRAND\nKANSAS CITY, MO 64108\n(39.076060142...",N,NaN,NaN
4,100036538,05/17/2010,17:17,03/15/2010,12:00,NaN,NaN,640,23F,Stealing From Auto,...,U,1.0,SUS,U,U,NaN,NaN,N,NaN,NaN


### Store Dataframe as File
Store combined original crime data (2010-2019) in a file to preserve original formatting and data quality

In [5]:
crimes.to_csv('crimes_2010-2019.csv',index=False)

### Import Complete Data Table (Crimes)

In [6]:
crimes_total = pd.read_csv('crimes_2010-2019.csv')

C:\Users\sjhal\Anaconda3\envs\dap\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,7,10,11,13,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Import Complete Data Table (Property Violations)

In [7]:
violations = pd.read_csv('kc_property_violations.csv')

C:\Users\sjhal\Anaconda3\envs\dap\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Assess Data

In [23]:
# investigate first entries in the table
crimes_total.head(20)

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,Beat,Address,Zip Code,City,Rep_Dist,Area,DVFlag,Invl_No,Involvement,Race,Sex,Age,Location,Firearm Used Flag,Latitude,Longitude
0,100048265,06/27/2010,13:46,06/27/2010,13:40,NaN,NaN,2655,90J,Trespassing,113,100 E 13 ST,NaN,KANSAS CITY,PJ1177,CPD,U,1.0,VIC,NaN,NaN,NaN,"100 13 ST\nKANSAS CITY, MO\n(39.19607633300046...",N,NaN,NaN
1,100043775,06/11/2010,21:00,06/10/2010,10:00,NaN,NaN,650,23G,Stealing Auto Parts/,999,9900 S UNKNOWN ST,0.0,KANSAS CITY,PKC000,PD1,U,1.0,VIC,B,M,40.0,NaN,N,NaN,NaN
2,100030602,04/27/2010,22:12,04/27/2010,19:00,NaN,NaN,1120,26A,Fraud,314,600 NORTON AV,64124.0,KANSAS CITY,PJ0731,EPD,U,1.0,VIC,W,F,59.0,"600 NORTON AV\nKANSAS CITY, MO 64124\n(39.1056...",N,NaN,NaN
3,100000611,01/03/2010,19:41,01/03/2010,19:41,NaN,NaN,1120,26A,Fraud,112,25/GRAND,64108.0,KANSAS CITY,PJ1015,CPD,U,2.0,VIC,NaN,NaN,NaN,"25 GRAND\nKANSAS CITY, MO 64108\n(39.076060142...",N,NaN,NaN
4,100036538,05/17/2010,17:17,03/15/2010,12:00,NaN,NaN,640,23F,Stealing From Auto,999,9900 S UNKNOWN ST,0.0,KANSAS CITY,PKC000,CPD,U,1.0,SUS,U,U,NaN,NaN,N,NaN,NaN
5,100040840,06/01/2010,21:04,06/01/2010,21:04,NaN,NaN,401,13A,Aggravated Assault (,314,E 6TH / ELMWOOD,64123.0,KANSAS CITY,PJ0796,EPD,N,1.0,SUS,B,M,NaN,NaN,N,NaN,NaN
6,100093508,12/01/2010,16:20,10/15/2010,3:30,11/30/2010,3:30,650,23G,Stealing Auto Parts/,NaN,UNKNOWN,NaN,KANSAS CITY,NaN,NaN,U,1.0,VIC,W,M,29.0,"UNKNOWN KANSAS CITY, MO\n(39.52368635500045, -...",N,NaN,NaN
7,100050190,07/03/2010,20:13,07/03/2010,19:57,NaN,NaN,690,23H,Stealing All Other,411,00 NE 50 CT,64118.0,KANSAS CITY,PC0863,NPD,Y,1.0,SUS,B,M,25.0,"00 50 CT\nKANSAS CITY, MO 64118\n(39.184582963...",N,NaN,NaN
8,100042964,06/09/2010,10:54,05/15/2010,12:00,NaN,NaN,650,23G,Stealing Auto Parts/,999,9900 S UNKNOWN ST,0.0,KANSAS CITY,PKC000,CPD,U,1.0,SUS,U,U,NaN,NaN,N,NaN,NaN
9,100046091,06/20/2010,1:55,06/20/2010,1:55,NaN,NaN,2655,90J,Trespassing,113,100 E 13 ST,64111.0,KANSAS CITY,PJ1177,CPD,U,1.0,VIC,NaN,NaN,NaN,"100 13 ST\nKANSAS CITY, MO 64111\n(39.05550278...",N,NaN,NaN


In [22]:
# investigate first entries in the table
violations.head()

,Property Violation ID,Case ID,Status,Case Opened Date,Case Closed Date,Days Open,Violation Code,Violation Description,Ordinance Number,Ordinance Chapter,Violation Entry Date,Address,County,State,Zip Code,Latitude,Longitude,KIVA PIN,Council District,Police Patrol Area,Inspection Area,Neighborhood,Code Violation Location
0,1225153,2019141615,Closed,07/30/2019,06/02/2020,62.0,NSWLOPSTOR,UNAPPROVED STORAGE,48-32 C.O.,48,08/06/2019,5111 E 40th St,Jackson,MO,64130.0,39.05270,-94.52384,32107,3.0,East,49,Vineyard,"5111 E 40th St\nMO 64130\n(39.0527, -94.52384)"
1,1059428,2012034662,Closed,03/21/2012,09/28/2012,140.0,NSWLLIMBS,LIMBS AND BRUSH,48-25 C.O.,48,03/22/2012,2300 E 74th St,Jackson,MO,64132.0,38.99297,-94.56039,115469,5.0,Metro,127,East Meyer 7,"2300 E 74th St\nMO 64132\n(38.99297, -94.56039)"
2,1124057,2014129843,Open,10/08/2014,NaN,2250.0,NSWLLIMBS,LIMBS AND BRUSH,48-25 C.O.,48,12/02/2020,7344 Lydia Ave,Jackson,MO,64131.0,38.99342,-94.57131,114703,5.0,Metro,128,East Meyer 6,"7344 Lydia Ave\nMO 64131\n(38.99342, -94.57131)"
3,1125358,2014139084,Closed,10/30/2014,06/01/2015,180.0,NSWLOPSTOR,UNAPPROVED STORAGE,48-32 C.O.,48,10/30/2014,2734 PASEO,Jackson,MO,64109.0,39.07638,-94.56570,27120,3.0,Central,25,Beacon Hills,"2734 PASEO\nMO 64109\n(39.07638, -94.5657)"
4,1162495,2016086040,Closed,07/25/2016,08/01/2016,7.0,NSWLLIMBS,LIMBS AND BRUSH,48-25 C.O.,48,07/26/2016,3005 E 32nd St,Jackson,MO,64128.0,39.06858,-94.54791,25056,3.0,East,57,Santa Fe,"3005 E 32nd St\nMO 64128\n(39.06858, -94.54791)"


In [10]:
# explore potential quality issues and data type for each variable
crimes_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234605 entries, 0 to 1234604
Data columns (total 26 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Report_No          1234605 non-null  object 
 1   Reported_Date      1234605 non-null  object 
 2   Reported_Time      1234605 non-null  object 
 3   From_Date          1232741 non-null  object 
 4   From_Time          1231871 non-null  object 
 5   To_Date            468955 non-null   object 
 6   To_Time            492664 non-null   object 
 7   Offense            1234605 non-null  object 
 8   IBRS               1210249 non-null  object 
 9   Description        1219662 non-null  object 
 10  Beat               1228952 non-null  object 
 11  Address            1109286 non-null  object 
 12  Zip Code           1202857 non-null  float64
 13  City               1109243 non-null  object 
 14  Rep_Dist           1201501 non-null  object 
 15  Area               1227404 non-n

In [12]:
crimes_total['Beat'].unique()

array([113.0, 999.0, 314.0, 112.0, nan, 411.0, 545.0, 242.0, 331.0, 243.0,
       142.0, 222.0, 234.0, 412.0, 642.0, 321.0, 312.0, 133.0, 231.0,
       232.0, 324.0, 213.0, 115.0, 341.0, 241.0, 122.0, 221.0, 322.0,
       224.0, 313.0, 542.0, 124.0, 413.0, 121.0, 415.0, 212.0, 544.0,
       344.0, 635.0, 343.0, 143.0, 334.0, 123.0, 543.0, 541.0, 643.0,
       421.0, 333.0, 141.0, 233.0, 134.0, 531.0, 535.0, 342.0, 422.0,
       332.0, 131.0, 132.0, 211.0, 223.0, 532.0, 345.0, 425.0, 644.0,
       114.0, 323.0, 636.0, 533.0, 641.0, 125.0, 534.0, 315.0, 631.0,
       634.0, 632.0, 311.0, 111.0, 144.0, 424.0, 414.0, 423.0, 316.0,
       633.0, 540.0, 385.0, 509.0, 137.0, 245.0, 216.0, 441.0, 325.0,
       109.0, 609.0, 675.0, 246.0, 537.0, 326.0, 139.0, 225.0, 564.0,
       346.0, 317.0, 2410.0, 0.0, 4999.0, 218.0, 336.0, 335.0, 220.0,
       209.0, 135.0, 244.0, 118.0, 309.0, 5411.0, 319.0, 235.0, 390.0,
       612.0, 621.0, 3321.0, 546.0, 3144.0, 145.0, 158.0, 671.0, 6410.0,
       146.

In [15]:
crimes_total['City'].value_counts().head(40)

KANSAS CITY          1102762
KCMO                    1828
INDEPENDENCE            1364
LIBERTY                  376
GRANDVIEW                360
RAYTOWN                  311
GLADSTONE                235
PLATTE CITY              185
LEES SUMMIT              149
N KANSAS CITY            106
PARKVILLE                101
OVERLAND PARK             80
KANAS CITY                78
BLUE SPRINGS              66
BELTON                    58
KEARNEY                   51
RIVERSIDE                 49
LEAVENWORTH               43
LEAWOOD                   39
PL                        36
PLEASANT VLY              36
NKC                       35
NORTH KANSAS CITY         33
HARRISONVILLE             30
SUGAR CREEK               28
SHAWNEE                   27
KANSAS                    25
KC                        24
LENEXA                    24
JA                        23
KASNAS CITY               19
INDEP                     19
MERRIAM                   17
KANASAS CITY              17
NORTHMOOR     

In [25]:
crimes_total['Zip Code'].value_counts().head(50)

64130.0    104211
64127.0     87413
64111.0     70522
64106.0     64034
64134.0     63244
64131.0     48793
64132.0     47914
64109.0     46192
64133.0     45752
64110.0     43632
64128.0     41612
64124.0     41573
64123.0     38516
64108.0     38337
64114.0     35414
64118.0     32418
64119.0     25869
64117.0     25363
64129.0     24921
64138.0     24719
64151.0     20591
64155.0     19527
64126.0     18548
64137.0     17653
64105.0     17128
99999.0     15711
64116.0     15379
64154.0     14443
64112.0     13392
64153.0     13001
64157.0     10975
64145.0     10060
64120.0      9525
64152.0      8906
64113.0      7505
64125.0      7498
64158.0      5372
64156.0      3411
64161.0      3352
64136.0      2792
64101.0      1946
64147.0      1796
64139.0      1494
64146.0      1492
64121.0       895
64102.0       890
0.0           477
64068.0       471
64104.0       437
64115.0       436
Name: Zip Code, dtype: int64

In [33]:
crimes_total[crimes_total.duplicated(subset="Report_No",keep=False)].sort_values("Reported_Date")

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,Beat,Address,Zip Code,City,Rep_Dist,Area,DVFlag,Invl_No,Involvement,Race,Sex,Age,Location,Firearm Used Flag,Latitude,Longitude
17860,100000186,01/01/2010,17:05,12/31/2009,22:00,01/01/2010,16:00,640,23F,Stealing From Auto,422,8600 N BOARDWALK AV,64153.0,KANSAS CITY,PP0321,NPD,U,1.0,SUS,U,U,NaN,"8600 BOARDWALK AV\nKANSAS CITY, MO 64153\n(39....",N,NaN,NaN
135358,100000124,01/01/2010,10:50,01/01/2010,10:30,NaN,NaN,802,13B,Non Agg Assault Dome,232,6600 LYDIA AV,64131.0,KANSAS CITY,PJ5070,MPD,U,1.0,VIC,B,F,29.0,"6600 LYDIA AV\nKANSAS CITY, MO 64131\n(39.0075...",N,NaN,NaN
134206,100000090,01/01/2010,6:56,01/01/2010,6:15,01/01/2010,6:20,702,240,Auto Theft,545,11300 DONNELLY AV,64134.0,KANSAS CITY,PJ7136,SPD,U,1.0,VIC,B,F,20.0,"11300 DONNELLY AV\nKANSAS CITY, MO 64134\n(38....",N,NaN,NaN
15047,100000026,01/01/2010,0:55,01/01/2010,0:44,NaN,NaN,1401,290,Property Damage,422,5100 NW 85 ST,64151.0,KANSAS CITY,PP0336,NPD,U,1.0,SUS,U,U,NaN,"5100 85 ST\nKANSAS CITY, MO 64151\n(39.2162419...",N,NaN,NaN
15098,100000246,01/01/2010,22:35,01/01/2010,10:00,01/01/2010,11:00,1401,290,Property Damage,341,6000 E 15 TE,64127.0,KANSAS CITY,PJ1431,EPD,U,1.0,SUS,U,U,NaN,"6000 15\nTE KANSAS CITY, MO 64127\n(39.0896241...",N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231604,KC19098977,12/31/2019,23:21,12/31/2019,23:21,NaN,NaN,Assault (Aggravated),13A,Aggravated Assault,333,4000 BELLEFONTAINE AVE,64127.0,KANSAS CITY,PJ3285,EPD,N,NaN,VIC,W,F,NaN,NaN,True,NaN,NaN
1231134,KC19098914,12/31/2019,16:18,12/31/2019,16:00,NaN,NaN,Trespass of Real Property,90J,Trespass of Real Property,643,8500 N CHURCH RD,64157.0,KANSAS CITY,PC0282,SCP,N,NaN,VIC,NaN,NaN,NaN,8500 N CHURCH RD\nKANSAS CITY 64157\n(39.2460...,False,NaN,NaN
1233213,KC19098804,12/31/2019,07:37,12/31/2019,07:37,NaN,NaN,Assault (Aggravated),13A,Aggravated Assault,213,6700 MANCHESTER AVE,64129.0,KANSAS CITY,PJ5274,MPD,N,NaN,SUS,W,M,36.0,NaN,False,NaN,NaN
1225851,KC19098948,12/31/2019,19:57,12/31/2019,19:57,12/31/2019,19:57,Assault (Non-Aggravated),13B,Simple Assault,332,35 ST and PROSPECT BLVD,64127.0,KANSAS CITY,NaN,EPD,N,NaN,VIC,B,M,81.0,NaN,False,NaN,NaN


In [34]:
crimes_total.query('Report_No == 100000172')

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,Beat,Address,Zip Code,City,Rep_Dist,Area,DVFlag,Invl_No,Involvement,Race,Sex,Age,Location,Firearm Used Flag,Latitude,Longitude
851,100000172,01/01/2010,15:00,01/01/2010,15:00,NaN,NaN,401,13A,Aggravated Assault (,314,SPRUCE AV/THOMPSON AV,64123.0,KANSAS CITY,PJ0502,EPD,U,1.0,VIC,W,M,20.0,"SPRUCE AV THOMPSON AV KANSAS CITY, MO 64123\n(...",Y,NaN,NaN
9597,100000172,01/01/2010,15:00,01/01/2010,15:00,NaN,NaN,401,13A,Aggravated Assault (,314,SPRUCE AV/THOMPSON AV,64123.0,KANSAS CITY,PJ0502,EPD,U,2.0,SUS,W,F,31.0,"SPRUCE AV THOMPSON AV KANSAS CITY, MO 64123\n(...",Y,NaN,NaN
47023,100000172,01/01/2010,15:00,01/01/2010,15:00,NaN,NaN,401,13A,Aggravated Assault (,314,SPRUCE AV/THOMPSON AV,64123.0,KANSAS CITY,PJ0502,EPD,U,2.0,VIC,W,F,21.0,"SPRUCE AV THOMPSON AV KANSAS CITY, MO 64123\n(...",Y,NaN,NaN
121831,100000172,01/01/2010,15:00,01/01/2010,15:00,NaN,NaN,401,13A,Aggravated Assault (,314,SPRUCE AV/THOMPSON AV,64123.0,KANSAS CITY,PJ0502,EPD,U,1.0,SUS,W,M,29.0,"SPRUCE AV THOMPSON AV KANSAS CITY, MO 64123\n(...",Y,NaN,NaN


In [11]:
# explore potential quality issues and data type for each variable
violations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777205 entries, 0 to 777204
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Property Violation ID    777205 non-null  int64  
 1   Case ID                  777205 non-null  int64  
 2   Status                   777205 non-null  object 
 3   Case Opened Date         777205 non-null  object 
 4   Case Closed Date         636925 non-null  object 
 5   Days Open                777129 non-null  float64
 6   Violation Code           777205 non-null  object 
 7   Violation Description    777205 non-null  object 
 8   Ordinance Number         777205 non-null  object 
 9   Ordinance Chapter        777205 non-null  int64  
 10  Violation Entry Date     777204 non-null  object 
 11  Address                  777200 non-null  object 
 12  County                   777023 non-null  object 
 13  State                    777201 non-null  object 
 14  Zip 

### Quality Issues
#### Crimes
- Convert dates and times to appropriate datetime format
- Remove unnecessary characters from the strings
- Convert columns (domestic violence, firearm used, zip code, description, etc) to categorical variables
- Delete data that is unnecessary for analysis due to missing data (To_Date, To_Time, Beat, Invl_No, Involvement)
- Eliminate redundancy of report number so that each crime only has one entry
- Correct city name incorrectly entered
- Remove "KC" prefix from later Report_No entries
- Delete entries for cities and zip codes outside of Kansas City, MO


#### Violations
- Format date columns to appropriate date time type
- Consolidate to data within 2010-2019
- Remove data missing zip code
- Clean "Case Opened Date" to reflect "Violation Entry Date" (in place of 12/05/2009)
- Remove redundant rows (Code Violation Location is covered by address, latitude, longitude)

In [70]:
crimes_total['Reported_Date'] = pd.to_datetime(crimes_total['Reported_Date'])

In [115]:
crimes_total['Zip Code'] = crimes_total['Zip Code'].astype('str').apply(lambda x: x[:5]).astype('category')
crimes_total['Zip Code'].value_counts().head(40)

64130    104211
64127     87413
64111     70522
64106     64034
64134     63244
64131     48793
64132     47914
64109     46192
64133     45752
64110     43632
64128     41612
64124     41573
64123     38516
64108     38337
64114     35414
64118     32418
nan       31748
64119     25869
64117     25363
64129     24921
64138     24719
64151     20591
64155     19527
64126     18548
64137     17653
64105     17128
99999     15711
64116     15379
64154     14443
64112     13392
64153     13001
64157     10975
64145     10060
64120      9525
64152      8906
64113      7505
64125      7498
64158      5372
64156      3411
64161      3352
Name: Zip Code, dtype: int64

In [80]:
crimes_total.groupby(['Reported_Date','Zip Code']).count()

Report_No  Reported_Time  From_Date  From_Time  \
Reported_Date Zip Code                                                   
01/01/2010    54130.0           1              1          1          1   
              64105.0           9              9          9          9   
              64106.0          27             27         27         27   
              64108.0          22             22         22         22   
              64109.0          19             19         19         19   
...                           ...            ...        ...        ...   
12/31/2019    64157.0           5              5          5          5   
              64165.0           1              1          1          1   
              64492.0           2              2          2          2   
              64701.0           1              1          1          1   
              nan              38             38         38         38   

                        To_Date  To_Time  Offense  IBRS  Description  Beat  \
Reported_Date Zip Code                                                       
01/01/2010    54130.0         0        0        1     1            1     1   
              64105.0         0        0        9     9            9     9   
              64106.0         2        2       27    27           27    27   
              64108.0         2        2       22    22           22    22   
              64109.0         4        4       19    19           19    19   
...                         ...      ...      ...   ...          ...   ...   
12/31/2019    64157.0         0        0        5     5            5     5   
              64165.0         1        1        1     0            0     1   
              64492.0         2        2        2     0            0     2   
              64701.0         0        0        1     0            0     1   
              nan             9        9       38    22           22    38   

                        ...  DVFlag  Invl_No  Involvement  Race  Sex  Age  \
Reported_Date Zip Code  ...                                                 
01/01/2010    54130.0   ...       1        1            1     1    1    1   
              64105.0   ...       9        9            9     5    5    5   
              64106.0   ...      27       27           27    21   21   16   
              64108.0   ...      22       22           22    19   19   16   
              64109.0   ...      19       19           19    17   17   11   
...                     ...     ...      ...          ...   ...  ...  ...   
12/31/2019    64157.0   ...       5        0            5     4    4    4   
              64165.0   ...       1        0            1     0    0    0   
              64492.0   ...       2        0            2     2    2    2   
              64701.0   ...       1        0            1     1    1    1   
              nan       ...      38        0           38    35   35   33   

                        Location  Firearm Used Flag  Latitude  Longitude  
Reported_Date Zip Code                                                    
01/01/2010    54130.0          1                  1         0          0  
              64105.0          9                  9         0          0  
              64106.0         27                 27         0          0  
              64108.0         22                 22         0          0  
              64109.0         19                 19         0          0  
...                          ...                ...       ...        ...  
12/31/2019    64157.0          5                  5         0          0  
              64165.0          1                  1         0          0  
              64492.0          2                  2         0          0  
              64701.0          1                  1         0          0  
              nan             34                 38         0          0  

[125882 rows x 24 columns]

In [94]:
violations['Violation Entry Date'] = pd.to_datetime(violations['Violation Entry Date'])

In [95]:
violations.groupby(['Violation Entry Date','Zip Code']).count()

Property Violation ID  Case ID  Status  \
Violation Entry Date Zip Code                                           
2008-05-12           64109.0                       4        4       4   
2009-05-07           64130.0                       3        3       3   
2009-08-07           64110.0                       1        1       1   
2009-08-11           64109.0                       5        5       5   
2009-10-08           64110.0                       1        1       1   
...                                              ...      ...     ...   
2020-12-04           64131.0                      28       28      28   
                     64132.0                      27       27      27   
                     64134.0                       8        8       8   
                     64137.0                       2        2       2   
                     64145.0                       1        1       1   

                               Case Opened Date  Case Closed Date  Days Open  \
Violation Entry Date Zip Code                                                  
2008-05-12           64109.0                  4                 4          4   
2009-05-07           64130.0                  3                 3          3   
2009-08-07           64110.0                  1                 1          1   
2009-08-11           64109.0                  5                 5          5   
2009-10-08           64110.0                  1                 1          1   
...                                         ...               ...        ...   
2020-12-04           64131.0                 28                 0         28   
                     64132.0                 27                 2         27   
                     64134.0                  8                 0          8   
                     64137.0                  2                 0          2   
                     64145.0                  1                 0          1   

                               Violation Code  Violation Description  \
Violation Entry Date Zip Code                                          
2008-05-12           64109.0                4                      4   
2009-05-07           64130.0                3                      3   
2009-08-07           64110.0                1                      1   
2009-08-11           64109.0                5                      5   
2009-10-08           64110.0                1                      1   
...                                       ...                    ...   
2020-12-04           64131.0               28                     28   
                     64132.0               27                     27   
                     64134.0                8                      8   
                     64137.0                2                      2   
                     64145.0                1                      1   

                               Ordinance Number  Ordinance Chapter  ...  \
Violation Entry Date Zip Code                                       ...   
2008-05-12           64109.0                  4                  4  ...   
2009-05-07           64130.0                  3                  3  ...   
2009-08-07           64110.0                  1                  1  ...   
2009-08-11           64109.0                  5                  5  ...   
2009-10-08           64110.0                  1                  1  ...   
...                                         ...                ...  ...   
2020-12-04           64131.0                 28                 28  ...   
                     64132.0                 27                 27  ...   
                     64134.0                  8                  8  ...   
                     64137.0                  2                  2  ...   
                     64145.0                  1                  1  ...   

                               County  State  Latitude  Longitude  KIVA PIN  \
Violation Entry Date Zip Code                   

In [107]:
valid_zips = list(violations['Zip Code'].dropna().unique())

In [93]:
violations['Zip Code'].value_counts()

64130.0    110216
64127.0     81733
64132.0     73389
64128.0     57163
64123.0     44556
64131.0     43851
64124.0     41433
64110.0     40944
64109.0     39541
64134.0     39169
64114.0     20807
64111.0     19577
64126.0     19539
64129.0     16388
64138.0     15336
64119.0     14192
64117.0     12724
64133.0     12141
64108.0     11274
64125.0      8312
64137.0      7929
64118.0      6314
64116.0      6196
64155.0      5936
64151.0      4902
64112.0      3297
64106.0      3073
64152.0      2619
64113.0      2436
64120.0      2224
64157.0      1426
64102.0      1386
64145.0      1154
64154.0      1114
64153.0       838
64136.0       787
64156.0       650
64105.0       498
64146.0       455
64101.0       413
64139.0       382
64158.0       305
64161.0       142
64163.0       109
64149.0        64
64164.0        64
64147.0        51
64166.0        26
64165.0        21
64160.0         1
64167.0         1
Name: Zip Code, dtype: int64